In [1]:
import simpy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
from collections import deque
plt.rcParams["figure.figsize"] = (10,10)

### Se está diseñando un web service, el cual cada vez que es invocado consulta a una base de datos.
### Se estima que el tiempo que transcurre entre cada llamada al servicio se puede modelar según una distribución exponencial con media 𝜇 = 4 𝑠𝑒𝑔𝑢𝑛𝑑𝑜𝑠
### Se debe decidir la arquitectura de base de datos a utilizar entre las dos siguientes:
### 1) Utilizar 2 bases de datos distribuidas. Con probabilidad 𝑝 = 0.6 las solicitudes son atendidas por la base 1 y con probabilidad 𝑞 = 1 − 𝑝 son atendidos por la base de datos 2. El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias, 𝜇1 = 0,7 𝑠𝑒𝑔 y 𝜇2 = 1 𝑠𝑒𝑔 respectivamente.
### 2) Utilizar 1 base de datos central. En este caso la demora en resolver una solicitud sigue una distribución exponencial con 𝜇 = 0,8 𝑠𝑒𝑔𝑢𝑛𝑑𝑜𝑠 
### Simular para cada opción 100000 solicitudes procesadas, determinando:
####  - El tiempo medio de espera entre que la solicitud llega y puede ser procesada (suponer que ninguna conexión cae por timeout).
####  - La fracción de las solicitudes que no esperaron para ser procesadas.
####  - La opción 1 es más costosa que la segunda opción y la empresa sólo acepta realizar la inversión si el tiempo medio que demora en resolver cada solicitud (tiempo en fila + tiempo de procesamiento) es como mínimo 50% menor que la opción 2. ¿Qué solución le recomienda?

In [13]:
class web_service_1:
    def __init__(self, env, media_call, bdd_1, bdd_2, proba_b1, media_1, media_2, sol_max):
        self.env = env
        self.media_call = media_call
        self.bdd_1 = bdd_1
        self.bdd_2 = bdd_2
        self.proba_b1 = proba_b1
        self.media_1 = media_1
        self.media_2 = media_2
        self.sol_max = sol_max
        self.action = env.process(self.run(env))
        self.usos_bdd = [0,0]
        self.cant_sol = 0
    
    def run(self, env):
        while(self.cant_sol < self.sol_max):
            yield self.env.timeout(sc.stats.expon.rvs(loc=0,scale=self.media_call))
            self.cant_sol += 1
            if (np.random.rand() < self.proba_b1):
                self.usos_bdd[0] += 1
                with self.bdd_1.request() as bdd1:
                    yield bdd1
                    yield self.env.timeout(sc.stats.expon.rvs(loc=0, scale=self.media_1))
            else:
                self.usos_bdd[1] += 1
                with self.bdd_2.request() as bdd2:
                    yield bdd2
                    yield self.env.timeout(sc.stats.expon.rvs(loc=0, scale=self.media_2))
                    
        print("Transcurrieron", self.env.now, "segundos en", self.sol_max, "iteraciones")         
        print("Se uso", self.usos_bdd[0], "veces la bdd1 y",  self.usos_bdd[1], "veces la bdd2")         

env = simpy.Environment()
base1 = simpy.Resource(env, capacity=1)
base2 = simpy.Resource(env, capacity=1)
ws1 = web_service_1(env, 4, base1, base2, 0.6, 0.7, 1, 100000)
env.run()

Transcurrieron 481384.18188283884 segundos en 100000 iteraciones
Se uso 60061 veces la bdd1 y 39939 veces la bdd2


In [14]:
class web_service_2:
    def __init__(self, env, media_call, bdd, media_bdd, sol_max):
        self.env = env
        self.media_call = media_call
        self.bdd = bdd
        self.media_bdd = media_bdd
        self.sol_max = sol_max
        self.action = env.process(self.run(env))
        self.cant_sol = 0
    
    def run(self, env):
        while(self.cant_sol < self.sol_max):
            yield self.env.timeout(sc.stats.expon.rvs(loc=0,scale=self.media_call))
            self.cant_sol += 1
            with self.bdd.request() as bdd:
                yield bdd
                yield self.env.timeout(sc.stats.expon.rvs(loc=0, scale=self.media_bdd))
                    
        print("Transcurrieron", self.env.now, "segundos en", self.sol_max, "iteraciones")


env = simpy.Environment()
base = simpy.Resource(env, capacity=1)
ws2 = web_service_2(env, 4, base, 0.8, 100000)
env.run()

Transcurrieron 478723.4394779483 segundos en 100000 iteraciones
